In [1]:
import pickle
from galmet.galmet import Galmet
import galmet.genetictransformer as gt

## Load Transformer Encoder models

In [ ]:
masker = gt.load_masker("../models/roberta-satire-v0.1/")
regression = gt.load_regression("../models/regression-v0.2/")

## Create GALMET model


In [ ]:
galmet = Galmet(masker=masker, regression=regression,
                crossover_prob = 0.2,
                total_mutation_prob = 0.8,
                population_size = 50,
                nb_generations = 50,
                goal_fitness = .99,
                max_edit_distance = 5,
                max_elites = 6,
                elite_duplicates = 3,
                mutate_token_prob = 0.7,
                add_token_prob = 0.05,
                remove_token_prob = 0.05)

In [ ]:
sentence = "One Number to Gauge Where the Economy Is Headed"
best, hof, logbook = galmet.evolve_text(sentence, print_logbook=True, print_final_population=True)

print("\n\nChosen:", best)

print("\n\nHall of fame (= Pareto Front):")
for h in reversed(hof):
    edit_distance = abs(-h.fitness.values[1])
    print(int(edit_distance), '%f' % h.fitness.values[0], masker.detokenize_sentence(h))

with open("./logbook.pickle", 'wb') as pickle_out:
    pickle.dump(logbook, pickle_out)

In [ ]:
with  open("./logbook.pickle", 'rb') as pickle_in:
    print("Read logbook from pickle")
    logbook = pickle.load(pickle_in)

In [ ]:
gen = logbook.select("gen")
# fit_mins = logbook.chapters["fitness"].select("min")
fit_avg = logbook.chapters["fitness"].select("avg")
fit_max = logbook.chapters["fitness"].select("max")
dist_avgs = logbook.chapters["distance"].select("avg")
dist_max = logbook.chapters["distance"].select("max")

import matplotlib.pyplot as plt

fig, ax1 = plt.subplots()
fig.set_size_inches(18.5, 10.5)

# Plot size
ax2 = ax1.twinx()
line_dist_avg = ax2.plot(gen, dist_avgs, "b-", label="Average Edit Distance")
line_dist_max = ax2.plot(gen, dist_max, "g-", label="Max Edit Distance")
ax2.set_ylabel("Edit Distance", color="r")
# for tl in ax2.get_yticklabels():
#     tl.set_color("r")

# Plot fitness
line1 = ax1.plot(gen, fit_max, "m-", label="Maximum Fitness")
line2 = ax1.plot(gen, fit_avg, "r-", label="Average Fitness")
# line3 = ax1.plot(gen, fit_mins, "b-", label="Minimum Fitness")
ax1.set_xlabel("Generation")
ax1.set_ylabel("Fitness", color="b")
# for tl in ax1.get_yticklabels():
#     tl.set_color("b")


lns = line_dist_avg + line_dist_max + line1 + line2
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc="upper left")

plt.show()

